In [ ]:
import torch
from torch.utils import data
import lightning as L
from lightning import Fabric
from lightning.pytorch import loggers as pl_loggers
from einops import rearrange

In [ ]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

In [ ]:
%load_ext autoreload
%autoreload 2
import data_module
from model import ImageClassification, OodClassification, MyModel

In [ ]:
# # Load dataset
train_dict = data_module.get_cifar10_far(is_train=True)
test_dict = data_module.get_cifar10_far(is_train=False)

train_data = train_dict["dataset"]
test_data = test_dict["dataset"]
num_classes = train_dict["num_classes_id"]

train_loader = data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)   # TODO change to train_data
test_loader = data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
len(train_data), len(test_data), len(train_loader), len(test_loader)

In [ ]:
# Create model
image_model = ImageClassification(num_classes=num_classes)
ood_model = OodClassification()
model = MyModel(image_model, ood_model)

In [ ]:
# Train model
trainer = L.Trainer(max_epochs=100, logger=True, devices=[2])
trainer.fit(model, train_loader, test_loader)

In [ ]:
y_hat = torch.rand((16, 2))
y = torch.randint(0, 2, (16,))
print(y_hat.dtype)
print(y.dtype)
ood_model.criterion(y_hat, y)